<a href="https://colab.research.google.com/github/itsZENR/RL_DQN/blob/main/RL_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 13.4 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import random
import asyncio

from tensorflow import device
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.losses import MeanSquaredError
from tensorflow.python.keras.initializers import RandomUniform
from collections import deque

In [40]:
env = gym.make("Acrobot-v1") # Загружаем среду
state = env.reset() # Получаем текущие состояние

REPLAY_MEMORY_SIZE = 50_000 # максимальное количиство данных для обучение ии
MIN_REPLAY_MEMORY_SIZE = 1000 # минимальное количиство данных для обучение ии
UPDATE_TARGET_EVERY = 5 # Через сколько будут обновлятся веса model_target
NUM_EPIZODS = 10 # Всево эпизодоа
max_steps = 10_000 # максимальное количиство шагов в эпизоде

In [52]:
class DQNAgent:

    def __init__(self):
      self.N_action = 2
      self.model = self.create_model()
      self.target_model = self.create_model()
      self.target_model.set_weights(self.model.get_weights())
      self.memory = deque(maxlen=REPLAY_MEMORY_SIZE)
      self.epsilon = 1.0
      self.epsilon_min = 0.001
      self.decay_epsilon = -0.0005
      self.gamma = 0.98
      self.update_target = 0

    def create_model(self):
        model = Sequential([
            Dense(100, activation='relu', kernel_initializer=RandomUniform(minval=-0.3, maxval=0.3)),
            Dense(50, activation='relu', kernel_initializer=RandomUniform(minval=-0.3, maxval=0.3)),
            Dense(2, activation='linear')
        ])
        model.compile(optimizer='adam')
        model.predict(state)
        return model

    def update_replay_memory(self, state, state_next, reward, action, done):
      self.memory.append([state, state_next, reward, action, done])

    def act(self, state):
      if random.random() < self.epsilon:
        return random.choice(range(3))
      else:
        action = self.model.predict(state)
        return np.argmax(action)

    def epsilon_minval(self):
      if self.epsilon > self.epsilon_min:
        self.epsilon = np.exp(self.decay_epsilon - self.epsilon)
      else:
        self.epsilon = self.epsilon_min

    async def train(self):
        if len(self.memory) < MIN_REPLAY_MEMORY_SIZE:
            return

        queue = asyncio.Queue()
        minibatch = random.sample(self.memory, len(self.memory))
        task = []

        for state, state_next, reward, action, done in minibatch:
          task.append(self.train_precess(state, state_next, reward, action, done))

        await queue.join()
        await asyncio.gather(*task, return_exceptions=True)

        self.update_target += 1

    def update_weights_target_model(self):
      if self.update_target == UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.update_target = 0

    async def train_precess(self, state, state_next, reward, action, done):
      if done:
          target = reward
      else:
          target = reward + self.gamma * np.max(self.model.predict(np.array(state)))
          target_q = self.target_model.predict(np.array(state_next))
          target_q[action] = target
          await self.model.fit(np.array(state), np.array(target_q), batch_size=len(self.memory), verbose=1)

In [55]:
agent = DQNAgent()
victory = 0

with device('/gpu:0'):
  for epizod in range(NUM_EPIZODS):
      state = env.reset()

      for step in range(max_steps):
        action = agent.act(state)
        if action > 2:action = 2
        observation, reward, done, info, _ = env.step(int(action))
        # print(f'action = {action}, step = {step}')
        agent.update_replay_memory(state, observation, reward, action, done)
        agent.train()
        agent.update_weights_target_model()
        agent.epsilon_minval()

        state = observation
        # print(f'reward = {reward}')

        if done:
          victory += 1
          print('max_step', step)
          print(f'victory = {victory}')
          break

<ipython-input-55-a7d2409e8228>:14: RuntimeWarning: coroutine 'DQNAgent.train' was never awaited
  agent.train()
Consider rewriting this model with the Functional API.


max_step 1216
victory = 1
max_step 1198
victory = 2
max_step 756
victory = 3
max_step 601
victory = 4
max_step 1164
victory = 5
max_step 1303
victory = 6
max_step 1281
victory = 7
max_step 1256
victory = 8
max_step 1183
victory = 9
max_step 1514
victory = 10


In [43]:
env.close()